### Set working directory 

In [ ]:
# Change to the data directory
import os 
new_directory = '/home/spunion/folding/data/re-sim/4'
os.chdir(new_directory)
print("cwd",os.getcwd())

### Streamlined Simulation

In [ ]:
! echo 14 | gmx pdb2gmx -f 1bz4.pdb -o 1bz4_processed.gro -water spce -ff "charmm27"

In [ ]:
! gmx editconf -f 1bz4_processed.gro -o 1bz4_box.gro -c -d 1.0 -bt cubic

In [ ]:
! gmx solvate -cp 1bz4_box.gro -cs spc216.gro -o 1bz4_solv.gro -p topol.top

In [ ]:
! touch ions.mdp

In [ ]:
! gmx grompp -f ions.mdp -c 1bz4_solv.gro -p topol.top -o ions.tpr -maxwarn 100

In [ ]:
! echo 13 |  gmx genion -s ions.tpr -o 1bz4_solv_ions.gro -p topol.top -pname NA -nname CL -neutral

In [ ]:
! gmx grompp -f emin.mdp -c 1bz4_solv_ions.gro -p topol.top -o emin.tpr -maxwarn 100
! gmx mdrun -v -deffnm emin

In [ ]:
! gmx grompp -f nvt.mdp -c emin.gro -r emin.gro -p topol.top -o nvt.tpr -maxwarn 100
! gmx mdrun -v -deffnm nvt

In [ ]:
! gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr -maxwarn 100
! gmx mdrun -v -deffnm npt

In [ ]:
! gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr -maxwarn 100
! gmx mdrun -v -deffnm md

In [ ]:
!printf "1\n0\n" | gmx trjconv -s emin.tpr -f md.xtc -o md_center.xtc -center -pbc mol

### Prepare Visualization Files

Use the output of the following cell to determine the last frame number. 

In [ ]:
! gmx check -f md.xtc

In [ ]:
# Include last frame number after -dump
! echo 0 | gmx trjconv -s emin.tpr -f md.xtc -o final_structure.gro -dump 500

### Run the following in jupyter notebook 

In [ ]:
# open jupyter notebook 
# ! jupyter notebook --allow-root

In [ ]:
import nglview as ng 
import mdtraj as md
traj = md.load("md_center.xtc", top="final_structure.gro")
view = ng.show_mdtraj(traj)
view